In [1]:
import pandas as pd
import numpy as np
import pybaseball
from pybaseball import statcast_batter
from pybaseball import playerid_lookup

# https://github.com/jldbc/pybaseball

### Transform Rank Shift Table from Statcast Website

In [9]:
#s = pd.read_excel('./Shift_leaders.xlsx') # got from statcast directly, copied table into excel sheet... could have web scrapped, but didn't think it was worth it for one table
#s['last'] = s['Name'].str.lstrip('\xa0').str.split(', ').str[0]
#s['first'] = s['Name'].str.lstrip('\xa0').str.split(', ').str[1]

#lst = []
#for i in range(s.shape[0]):
    #print(s.iloc[i]['last'])
#    lst.append(playerid_lookup(s.iloc[i]['last'], s.iloc[i]['first'], fuzzy=True)['key_mlbam'].values[0])
    
#s = pd.concat([s, pd.Series(lst, name='key')], axis=1)

#top_shifted_players_with_mlbam = s

#top_shifted_players_with_mlbam.to_csv('./top_shifted_players_with_mlbam.csv', index=False)

top_shifted_players_with_mlbam = pd.read_csv('./data/top_shifted_players_with_mlbam.csv')

### All Statcast Data from Ranked Players 2022

In [10]:
#lst_df = []
#for i in range(s.shape[0]):
#    lst_df.append(statcast_batter('2022-04-01', '2022-09-30', player_id = s.iloc[i]['key']))
#stat_df = pd.concat(lst_df, axis=0, ignore_index=True)
#stat_df.to_csv('./statcast_dataset_2022.csv', index=False)

statcast_dataset_2022 = pd.read_csv('./data/statcast_dataset_2022.csv')

### All Statcast Data from Ranked Players 2023

In [11]:
#lst_df2 = []
#for i in range(s.shape[0]):
#    lst_df2.append(statcast_batter('2023-04-01', '2023-09-30', player_id = s.iloc[i]['key']))
#stat_df2 = pd.concat(lst_df2, axis=0, ignore_index=True)
#stat_df2.to_csv('./statcast_dataset_2023.csv', index=False)

statcast_dataset_2023 = pd.read_csv('./data/statcast_dataset_2023.csv')

### Merge above Tables Together

In [12]:
statcast_dataset = pd.concat([statcast_dataset_2022, statcast_dataset_2023], axis=0)

df = pd.merge(statcast_dataset, top_shifted_players_with_mlbam, left_on='batter', right_on='key', how='left')

### Find All Players with > 162 ABs in 2022 and 2023

In [13]:
cnt_2022 = []
for x in statcast_dataset_2022.player_name.unique():
    y = len(statcast_dataset_2022[statcast_dataset_2022['player_name'] == x][['game_date', 'at_bat_number']].drop_duplicates())
    #print(x, y)
    if y > 162:
        cnt_2022.append(x)
#print(cnt_2022)


cnt_2023 = []
for x in statcast_dataset_2023.player_name.unique():
    y = len(statcast_dataset_2023[statcast_dataset_2023['player_name'] == x][['game_date', 'at_bat_number']].drop_duplicates())
    #print(x, y)
    if y > 162:
        cnt_2023.append(x)
#print(cnt_2023)

valid_players = [x for x in cnt_2022 if x in cnt_2023]

valid_players
    

['Santana, Carlos',
 'Raleigh, Cal',
 'Seager, Corey',
 'Calhoun, Kole',
 'Gallo, Joey',
 'Pasquantino, Vinnie',
 'Hicks, Aaron',
 'Tucker, Kyle',
 'Alvarez, Yordan',
 'Brown, Seth',
 'Schwarber, Kyle',
 'Kepler, Max',
 'Ohtani, Shohei',
 'Bellinger, Cody',
 'Polanco, Jorge',
 'Lowe, Brandon',
 'Rizzo, Anthony',
 'Albies, Ozzie',
 'Muncy, Max',
 'Votto, Joey',
 'Santander, Anthony',
 'Rosario, Eddie',
 'Kelenic, Jarred',
 'Massey, Michael',
 'Kirilloff, Alex',
 'Belt, Brandon',
 'Rutschman, Adley',
 'Heim, Jonah',
 'Biggio, Cavan',
 'Grossman, Robbie',
 'Moustakas, Mike',
 'Tellez, Rowdy',
 'Olson, Matt',
 'Bleday, JJ',
 'Ruiz, Keibert',
 'Naylor, Josh',
 'Melendez, MJ',
 'Yastrzemski, Mike',
 'Pratto, Nick',
 'Grandal, Yasmani',
 'Winker, Jesse',
 'Suárez, Eugenio',
 'Fraley, Jake',
 'Vogelbach, Daniel',
 'Pederson, Joc',
 'Chisholm Jr., Jazz',
 'Escobar, Eduardo',
 'Buxton, Byron',
 'Moncada, Yoán',
 'Soler, Jorge',
 'Harper, Bryce',
 'Reynolds, Bryan',
 'Perez, Salvador',
 'Blackmon

### Final Dataset Transforms

In [14]:
df = df[df['player_name'].isin(valid_players)]

df['shift_rank'] = df.Ranking.rank(method='dense')

df.drop(['Plate Apperances', 'Ranking'], axis=1, inplace=True)

In [15]:
# include batter/pitchers teams
df['bat_team'] = df.apply(lambda row: row['home_team'] if row['inning_topbot'] == 'Bot' else row['away_team'], axis=1)
df['pitch_team'] = df.apply(lambda row: row['away_team'] if row['inning_topbot'] == 'Bot' else row['home_team'], axis=1)

In [16]:
# column projection
df = df[['game_date', 'batter', 'pitcher', 'stand', 'bb_type', 'hc_x', 'hc_y', 'hit_distance_sc', 'launch_speed', 'shift_rank', 'bat_team', 'pitch_team']]

# keep only rows where ball is in play
df = df[~df['bb_type'].isna()]

# center, reorient y-direction
df['hc_x'] = df['hc_x'] - 126
df['hc_y'] = df['hc_y'] - 204
df['hc_y'] = - 1 * df['hc_y']

def delete_foul_balls(x, y):
    if y < 0:
        return False
    if x > 0:
        if x > y:
            return False
    else:
        if -x > y:
            return False
    return True

# only keep balls that landed in play
df['angle'] = df.apply(lambda row: delete_foul_balls(row['hc_x'], row['hc_y']), axis=1)
df = df[df['angle']]
df.drop('angle', axis=1, inplace=True)

# calculate angle from first base line
import math
def angle(x, y):
    if x > 0:
        return math.degrees(math.atan(y / x)) - 45
    elif x < 0:
        return 135 + math.degrees(math.atan(y / x))
    else:
        return 45
df['angle_from_first_base_line'] = df.apply(lambda row: angle(row['hc_x'], row['hc_y']), axis=1)

# drop rows that include null values (< 1% of all data)
# mostly ball speed and distance, SC doesn't have 100% coverage on these stats...
df = df.dropna()

In [17]:
col_names = {
    'game_date':'date', 
    'batter':'batter_SC_ID', 
    'pitcher':'pitcher_SC_ID', 
    'stand':'batter_handedness', 
    'bb_type':'batted_ball_type', 
    'hc_x':'x', 
    'hc_y':'y',
    'hit_distance_sc':'dist', 
    'launch_speed':'launch_speed', 
    'shift_rank':'shift_rank',
    'angle_from_first_base_line':'angle_from_first_base_line'
}

df.rename(columns=col_names, inplace=True)

### Final Dataset

In [18]:
df.head()
# https://baseballsavant.mlb.com/csv-docs
# glossery of stat meanings

,date,batter_SC_ID,pitcher_SC_ID,batter_handedness,batted_ball_type,x,y,dist,launch_speed,shift_rank,bat_team,pitch_team,angle_from_first_base_line
209,2022-09-30,467793,686610,R,line_drive,-9.78,180.30,389.0,109.7,1.0,SEA,OAK,48.104848
213,2022-09-30,467793,686610,R,ground_ball,-20.11,52.93,44.0,107.8,1.0,SEA,OAK,65.803575
229,2022-09-28,467793,527048,R,ground_ball,7.77,59.52,10.0,110.8,1.0,SEA,TEX,37.562419
232,2022-09-28,467793,527048,R,ground_ball,-6.43,37.21,2.0,70.1,1.0,SEA,TEX,54.804062
250,2022-09-27,467793,622786,L,fly_ball,22.96,146.68,355.0,100.2,1.0,SEA,TEX,36.103613


### Save dataset

In [19]:
df.sort_values(by=['shift_rank', 'date']).to_csv('./data/final_cleaned_statcast_shift_data.csv', index=False)


# cols we could use later...
# pitch_type
# zone (zone location of the ball when it crosses the plate from the catcher's persepective)
# delta run exp (the change in run expectancy before the pitch and after the pitch)